In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [3]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

2023-05-10 22:51:01.104317: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2023-05-10 22:52:04.455611: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
import os
os.getcwd()

"/Users/davidchu/Desktop/USC/Projects/Aiden's extened stats comps"

In [8]:
import pandas as pd
import numpy as np

fn = 'msft_tweets_clean.csv'
df=pd.read_csv(fn)

In [10]:
from datasets import load_dataset

dataset = load_dataset("zeroshot/twitter-financial-news-sentiment")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /Users/davidchu/.cache/huggingface/datasets/zeroshot___csv/zeroshot--twitter-financial-news-sentiment-ccca0f3c622c5b67/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
flattened_train_data = dataset['train'].flatten()
flattened_test_data =  dataset['validation'].flatten()
# Selecting a subset of the train dataset
train = pd.DataFrame(flattened_train_data.select(range(3000)))

# Selecting a subset of the test dataset
test = pd.DataFrame(flattened_test_data.select(range(1000)))

In [30]:
train = train.loc[train['label'] != 2]
test = test.loc[test['label'] != 2]

In [41]:
def give_emoji_free_text(text):
    """
    Removes emoji's from tweets
    Accepts:
        Text (tweets)
    Returns:
        Text (emoji free tweets)
    """
    emoji_list = [c for c in text if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

def url_free_text(text):
    '''
    Cleans text from urls
    '''
    text = re.sub(r'http\S+', '', text)
    return text

def get_lemmas(text):
        '''Used to lemmatize the processed tweets'''
        lemmas = []
        
        doc = nlp(text)
        
        # Something goes here :P
        for token in doc: 
            if ((token.is_stop == False) and (token.is_punct == False)) and (token.pos_ != 'PRON'):
                lemmas.append(token.lemma_)
        
        return lemmas

def tokenize(text):
        """
        Parses a string into a list of semantic units (words)
        Args:
            text (str): The string that the function will tokenize.
        Returns:
            list: tokens parsed out
        """
        # turn text into lower case
        tokens = text.lower()
        # removing hashtags and mentions
        tokens = re.sub(r"@[A-Za-z0-9_]+"," ", tokens)
        tokens = re.sub(r"#[A-Za-z0-9_]+"," ", tokens)
        # removing numbers
        tokens = re.sub(r"\b\d+\b"," ", tokens)
        # removing &amp
        tokens = re.sub(r"&amp|amp", " ", tokens)
        # removing urls
        tokens = re.sub(r"http\S+", " ", tokens)
        tokens = re.sub(r"www.\S+", " ", tokens)
        # removing punctuations 
        tokens = re.sub(r'[()!?]', " ", tokens)
        tokens = re.sub(r'\[.*?\]'," ", tokens)
        # removing non-alphanumeric characters
        tokens = re.sub(r'\W+'," ", tokens)
        # removing politcians' names
        tokens = re.sub(r'nancy|brown|cantwell|cardin|carper|casey|feinstein|klobuchar|menendez|sanders|stabenow|tester|whitehouse|barrasso|wicker|collins|cornyn|durbin|graham|inhofe|mcconnell|merkley|reed|risch|shaheen|warner|gillibrand|coons|manchin|aderholt|baldwin|bass|bennet|bilirakis|bishop of georgia|blackburn|blumenauer|blumenthal|blunt|boozman|brady|brooks|buchanan|bucshon|burgess|burr|butterfield|calvert|capito|carson|carter of texas|cassidy|castor|chabot|chu|cicilline|clarke|cleaver|clyburn|cohen|cole|connolly|cooper|costa|courtney|crapo|crawford|cuellar|danny davis of illinois|defazio|degette|delauro|desjarlais|diaz-balart|doggett|doyle|duncan|eshoo|fleischmann|foxx|garamendi|gibbs|gohmert|gosar|granger|grassley|graves of missouri|green of texas|griffith|grijalva|guthrie|harris|hartzler|heinrich|herrera beutler|higgins of new york|himes|hirono|hoeven|hoyer|huizenga|jackson lee|johnson of ohio|johnson of texas|johnson of georgia|johnson|jordan|kaptur|keating|kelly of pennsylvania|kind|kinzinger|lamborn|langevin|lankford|larsen|larson|latta|leahy|lee of california|lee|lofgren|long|lucas|luetkemeyer|luján|lynch|maloney of new york|markey|matsui|mccarthy|mccaul|mcclintock|mccollum|mcgovern|mchenry|mckinley|mcmorris rodgers|mcnerney|meeks|moore of wisconsin|moran|murkowski|murphy|murray|nadler|napolitano|neal|palazzo|pallone|pascrell|paul|pelosi|perlmutter|peters|pingree|portman|posey|price|quigley|rogers of kentucky|rogers of alabama|roybal-allard|rubio|ruppersberger|rush|ryan of ohio|sarbanes|scalise|schakowsky|schiff|schrader|schumer|schweikert|austin scott of georgia|david scott of georgia|scott of virginia|scott of south carolina|sewell|shelby|sherman|simpson|sires|smith of washington|smith of nebraska|smith of new jersey|speier|sánchez|thompson of mississippi|thompson of california|thompson of pennsylvania|thune|tonko|toomey|turner|upton|van hollen|velázquez|walberg|wasserman schultz|waters|webster|welch|wilson of south carolina|wilson of florida|wittman|womack|wyden|yarmuth|young|amodei|bonamici|delbene|massie|payne|schatz|foster|titus|cotton|sinema|lamalfa|huffman|bera|swalwell|brownley|cárdenas|ruiz|takano|lowenthal|vargas|peters|frankel|duckworth|rodney davis of illinois|bustos|barr|warren|king|kildee|wagner|daines|hudson|cramer|fischer|kuster|meng|jeffries|maloney of new york|wenstrup|beatty|joyce of ohio|mullin|perry|cartwright|rice of south carolina|cruz|weber|castro|williams of texas|veasey|stewart|kaine|kilmer|pocan|kelly of illinois|smith of missouri|booker|clark|norcross|adams|palmer|hill|westerman|gallego|desaulnier|aguilar|lieu|torres of california|buck|carter of georgia|hice|loudermilk|allen|bost|graves of louisiana|moulton|moolenaar|dingell|lawrence|emmer|rouzer|watson coleman|zeldin|rice of new york|stefanik|katko|boyle|babin|beyer|newhouse|grothman|mooney|sullivan|ernst|tillis|rounds|sasse|kelly of mississippi|lahood|davidson|comer|evans|kennedy|hassan|cortez masto|schneider|o’halleran|biggs|khanna|panetta|carbajal|barragán|correa|blunt rochester|gaetz|dunn|rutherford|lawson|murphy of florida|soto|demings|mast|ferguson|krishnamoorthi|banks|hollingsworth|marshall|higgins of louisiana|johnson of louisiana|brown of maryland|raskin|bergman|budd|bacon|gottheimer|rosen|suozzi|espaillat|fitzpatrick|smucker|kustoff|gonzalez of texas|arrington|mceachin|jayapal|gallagher|cheney|estes|norman|gomez|curtis|smith|hyde-smith|lamb|lesko|cloud|balderson|hern|morelle|scanlon|wild|case|horsford|kirkpatrick|stanton|harder|porter|levin of california|neguse|crow|hayes|waltz|steube|mcbath|axne|fulcher|garcía|casten|underwood|baird|pence|davids|trahan|pressley|trone|slotkin|levin of michigan|stevens|tlaib|craig|phillips|omar|stauber|guest|armstrong|pappas|van drew|kim of new jersey|malinowski|sherrill|lee of nevada|ocasio-cortez|gonzalez of ohio|dean|houlahan|meuser|joyce of pennsylvania|reschenthaler|timmons|johnson of south dakota|burchett|rose|green of tennessee|crenshaw|taylor|gooden|fletcher|escobar|roy|garcia of texas|allred|luria|cline|spanberger|wexton|schrier|steil|miller of west virginia|scott of florida|braun|hawley|romney|golden|keller|bishop of north carolina|murphy of north carolina|mfume|tiffany|garcia of california|jacobs of new york|kelly|lummis|issa|sessions|valadao|tuberville|hickenlooper|hagerty|carl|moore of alabama|obernolte|kim of california|steel|jacobs of california|boebert|cammack|franklin|donalds|gimenez|salazar|williams of georgia|bourdeaux|clyde|greene|kahele|hinson|miller-meeks|feenstra|newman|miller of illinois|mrvan|spartz|mann|laturner|auchincloss|meijer|mcclain|fischbach|bush|rosendale|ross|manning|cawthorn|herrell|leger fernandez|garbarino|malliotakis|torres of new york|bowman|jones|bice|bentz|mace|harshbarger|fallon|pfluger|jackson|nehls|gonzales|van duyne|moore of utah|owens|good|strickland|fitzgerald|padilla|ossoff|warnock|tenney|letlow|carter of louisiana|stansbury|ellzey|brown of ohio|carey|cherfilus-mccormick|conway|flores|flood|finstad|peltola|ryan of new york|sempolinski'," ", tokens)
        # removing state names
        tokens = re.sub(r'alabama|alaska|arizona|arkansas|california|colorado|connecticut|delaware|florida|georgia|hawaii|idaho|illinois|indiana|iowa|kansas|kentucky|louisiana|maine|maryland|massachusetts|michigan|minnesota|mississippi|missouri|montana|nebraska|nevada|new hampshire|new jersey|new mexico|new york|north carolina|north dakota|ohio|oklahoma|oregon|pennsylvania|rhode island|south carolina|south dakota|tennessee|texas|utah|vermont|virginia|washington|west virginia|wisconsin|wyoming'," ", tokens)
#         # split the text
#         tokens = tokens.split() 
#         # removing stop words
#         stop_words = open("./Data/stopwords.txt",'r').read().splitlines()
#         stop_words = [word.strip() for word in stop_words]
#         tokens = [w for w in tokens if not w in stop_words]
#         tokens = ' '.join(tokens)
    
        return tokens

def clean_text(df):
    '''
    get topics from list of documents
    '''
    #Create a new column with url free tweets
    df['url_free_tweets'] = df['text'].apply(url_free_text)
    # Tokenizer
    tokenizer = Tokenizer(nlp.vocab)
    # Custom stopwords
    custom_stopwords = ['hi','\n','\n\n', '&', ' ', '.', '-', 'got', "it's", 'it’s', "i'm", 'i’m', 'im', 'want', 'like', '$', '@']
    # Customize stop words by adding to the default list
    STOP_WORDS = nlp.Defaults.stop_words.union(custom_stopwords)
    # ALL_STOP_WORDS = spacy + gensim + wordcloud
    ALL_STOP_WORDS = STOP_WORDS.union(SW).union(stopwords)
    tokens = []
    for doc in tokenizer.pipe(df['url_free_tweets'], batch_size=500):
        doc_tokens = []    
        for token in doc: 
            if token.text.lower() not in ALL_STOP_WORDS:
                doc_tokens.append(token.text.lower())   
        tokens.append(doc_tokens)

    # Makes tokens column
    df['tokens'] = tokens
    # Make tokens a string again
    df['tokens_back_to_text'] = [' '.join(map(str, l)) for l in df['tokens']]
    df['lemmas'] = df['tokens_back_to_text'].apply(get_lemmas)
    # Make lemmas a string again
    df['lemmas_back_to_text'] = [' '.join(map(str, l)) for l in df['lemmas']]
    # Apply tokenizer
    df['lemma_tokens'] = df['lemmas_back_to_text'].apply(tokenize)

In [47]:
import emoji
import re
import os
import spacy
from spacy.tokenizer import Tokenizer
from gensim.parsing.preprocessing import STOPWORDS as SW
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)
nlp = spacy.load('en_core_web_lg')
clean_text(train)
clean_text(test)

In [49]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

  train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                           test, 
                                                                           'DATA_COLUMN', 
                                                                           'LABEL_COLUMN')
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'lemma_tokens'
LABEL_COLUMN = 'label'

In [50]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

/Users/davidchu/opt/anaconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [51]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)

Epoch 1/2
54/54 [==============================] - 646s 12s/step - loss: 0.3859 - accuracy: 0.8409 - val_loss: 0.4557 - val_accuracy: 0.8182
Epoch 2/2
54/54 [==============================] - 5638s 106s/step - loss: 0.1123 - accuracy: 0.9589 - val_loss: 0.5637 - val_accuracy: 0.8235


In [73]:
pred_sentences = ['JPMorgan lost a lot of money today',
                 'I do not want to buy this stock again']

In [74]:
tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['Negative','Positive']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
for i in range(len(pred_sentences)):
  print(pred_sentences[i], "\n", labels[label[i]])

JPMorgan lost a lot of money today 
 Negative
I do not want to buy this stock again 
 Negative
